<b style="color: red;">last scraped date: 05/2020</b>

Source: http://www.stats.gov.cn/english/PressRelease/ - National Statistics Bureuo of China<br>
<ul>
    <li>
        part 1: retrived url from history sqlite/ Firefox</li>
    <li>
        part 2: manual copmillation from the website</li>
</ul>

Missing data: 2015/09 August not published<br>

Saved html body in json in {'date':<body\>} format<br>

Note: website prone to throw 200 error so repetitive try needed.
        

In [1]:
import pandas as pd
import sqlite3 as sq3
import requests as rq
from bs4 import BeautifulSoup as beauty
import json

In [3]:
#get the pages from previous browser history
conn = sq3.connect('places.sqlite')
q = "select * from sqlite_master where type = 'table' "
df = pd.read_sql(q, conn)
df

,type,name,tbl_name,rootpage,sql
0,table,moz_places,moz_places,3,CREATE TABLE moz_places ( id INTEGER PRIMARY...
1,table,moz_historyvisits,moz_historyvisits,4,CREATE TABLE moz_historyvisits ( id INTEGER P...
2,table,moz_inputhistory,moz_inputhistory,5,CREATE TABLE moz_inputhistory ( place_id INTE...
3,table,moz_hosts,moz_hosts,7,CREATE TABLE moz_hosts ( id INTEGER PRIMARY K...
4,table,moz_bookmarks,moz_bookmarks,9,CREATE TABLE moz_bookmarks ( id INTEGER PRIMA...
5,table,moz_keywords,moz_keywords,10,CREATE TABLE moz_keywords ( id INTEGER PRIMAR...
6,table,sqlite_sequence,sqlite_sequence,12,"CREATE TABLE sqlite_sequence(name,seq)"
7,table,moz_anno_attributes,moz_anno_attributes,13,CREATE TABLE moz_anno_attributes ( id INTEGER...
8,table,moz_annos,moz_annos,15,CREATE TABLE moz_annos ( id INTEGER PRIMARY K...
9,table,moz_items_annos,moz_items_annos,16,CREATE TABLE moz_items_annos ( id INTEGER PRI...


In [4]:
q = "select url from moz_places where url like '%stats.gov.cn/english/PressRelease%.html'"
df = pd.read_sql(q, conn)
links = []
for i in df.values:
    links.append(i[0])

In [5]:
len(links)

53

In [2]:
#manually collected links from the stat website
links2 = ['http://www.stats.gov.cn/english/PressRelease/201909/t20190917_1698040.html',
          'http://www.stats.gov.cn/english/PressRelease/201908/t20190830_1694744.html',
          'http://www.stats.gov.cn/english/PressRelease/201907/t20190716_1676345.html',
          'http://www.stats.gov.cn/english/PressRelease/201906/t20190618_1671088.html',
          'http://www.stats.gov.cn/english/PressRelease/201905/t20190522_1666352.html',
          'http://www.stats.gov.cn/english/PressRelease/201904/t20190416_1659789.html',
          'http://www.stats.gov.cn/english/PressRelease/201903/t20190315_1654102.html',
          'http://www.stats.gov.cn/english/PressRelease/201902/t20190222_1650311.html',
          'http://www.stats.gov.cn/english/PressRelease/201901/t20190117_1645410.html',
          'http://www.stats.gov.cn/english/PressRelease/201812/t20181217_1639809.html',
          'http://www.stats.gov.cn/english/PressRelease/201811/t20181115_1633772.html',
          'http://www.stats.gov.cn/english/PressRelease/201810/t20181022_1629019.html',
          'http://www.stats.gov.cn/english/PressRelease/202002/t20200217_1726742.html',
          'http://www.stats.gov.cn/english/PressRelease/202001/t20200116_1723191.html',
          'http://www.stats.gov.cn/english/PressRelease/201912/t20191216_1717707.html',
          'http://www.stats.gov.cn/english/PressRelease/201911/t20191118_1709829.html',
          'http://www.stats.gov.cn/english/PressRelease/201910/t20191021_1704333.html',
          'http://www.stats.gov.cn/english/PressRelease/202004/t20200416_1739083.html',
          'http://www.stats.gov.cn/english/PressRelease/202003/t20200316_1732288.html',
          'http://www.stats.gov.cn/english/PressRelease/202005/t20200518_1745998.html'
         ]

update = [
          'http://www.stats.gov.cn/english/PressRelease/202005/t20200518_1745998.html'
         ]

In [3]:
#alllinks = links + links2
alllinks = update

In [4]:
headers = headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'}
dick = {}
broken = []
n=1
for i in alllinks:
    p = rq.get(i, headers = headers)
    if p.status_code == 200:
        soup = beauty(p.content, 'lxml')     
        dick[i.split('/')[5]] = str(soup.find('div', attrs = {'class': 'TRS_Editor'}) )
    else:
        broken.append(i)
    print(n)
    n+=1

print('broken links :',len(broken))
print('retrived pages :', len(dick))

# recursive download for failed connections
while len(broken)> 0:
    print('url left: {}'.format(len(broken)))
    linkcycle = broken[:]
    #headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'}
    broken = []
    n=1
    for i in linkcycle:
        p = rq.get(links2[-1], headers = headers)
        if p.status_code == 200:
            soup = beauty(p.content, 'lxml')
            dick[i.split('/')[5]] = str(soup.find('div', attrs = {'class': 'TRS_Editor'}))
        else:
            broken.append(i)
        print(n)
        n+=1

1
broken links : 0
retrived pages : 1


In [5]:
len(dick)

1

#save to json
with open('china_tables.json', 'w') as js:
    json.dump(dick, js)

In [7]:
#update with new scare
with open('china_tables.json','r') as js:
    d = json.load(js)
print(len(d))
with open('china_tables.json','w') as js:
    d.update(dick)
    json.dump(d, js)
print(d)

69


In [8]:
#check content size
with open('china_tables.json', 'r') as js:
    dump = json.load(js)
len(dump)    

70